In [41]:
import pandas as pd
print("Codespaces fonctionne !")

Codespaces fonctionne !


In [42]:
import os
os.chdir('/workspaces/projet-iadata')  # Change le répertoire vers la racine
print(os.getcwd())  # Vérif
#qlq soucis avec les chemins


/workspaces/projet-iadata


In [43]:
#IMPORTER LES DATASET
conso= pd.read_csv('./data/brut/consommation-quotidienne-brute.csv')

print(conso.head()) #colums
print(conso.info()) #infos de la columns
print(conso.describe())
#constat le fichier semble mal séparé (toute la ligne est import sur une meme colonne)

  Date - Heure;Date;Heure;Consommation brute gaz (MW PCS 0°C) - GRTgaz;Statut - GRTgaz;Consommation brute gaz (MW PCS 0°C) - Teréga;Statut - Teréga;Consommation brute gaz totale (MW PCS 0°C);Consommation brute électricité (MW) - RTE;Statut - RTE;Consommation brute totale (MW)
0  2024-04-30T21:30:00+00:00;30/04/2024;23:30;;;;...                                                                                                                                                                                                                                
1  2024-04-30T21:00:00+00:00;30/04/2024;23:00;225...                                                                                                                                                                                                                                
2  2024-04-30T20:30:00+00:00;30/04/2024;22:30;;;;...                                                                                                                     

## NETTOYAGE DE DONNEES

### VERIFIER LES SEPARATEURS

In [44]:
#use sep=";" pour mieux organisé le fichier
conso=pd.read_csv('./data/brut/consommation-quotidienne-brute.csv', sep=';')

#verif des colonnes
print(conso.head())
print(conso.info())
print(conso.describe())

                Date - Heure        Date  Heure  \
0  2024-04-30T21:30:00+00:00  30/04/2024  23:30   
1  2024-04-30T21:00:00+00:00  30/04/2024  23:00   
2  2024-04-30T20:30:00+00:00  30/04/2024  22:30   
3  2024-04-30T20:00:00+00:00  30/04/2024  22:00   
4  2024-04-30T19:30:00+00:00  30/04/2024  21:30   

   Consommation brute gaz (MW PCS 0°C) - GRTgaz Statut - GRTgaz  \
0                                           NaN             NaN   
1                                       22574.0       Définitif   
2                                           NaN             NaN   
3                                       24265.0       Définitif   
4                                           NaN             NaN   

   Consommation brute gaz (MW PCS 0°C) - Teréga Statut - Teréga  \
0                                           NaN             NaN   
1                                        1562.0       Définitif   
2                                           NaN             NaN   
3                     

##### RENOMMER LES COLONNNES

In [45]:
# conso.columns['Date','Heure','Consommation', 'source','Unité']
# print(conso.head())
print(conso.columns)


Index(['Date - Heure', 'Date', 'Heure',
       'Consommation brute gaz (MW PCS 0°C) - GRTgaz', 'Statut - GRTgaz',
       'Consommation brute gaz (MW PCS 0°C) - Teréga', 'Statut - Teréga',
       'Consommation brute gaz totale (MW PCS 0°C)',
       'Consommation brute électricité (MW) - RTE', 'Statut - RTE',
       'Consommation brute totale (MW)'],
      dtype='object')


In [46]:
#certains noms sont trop long
# Renommer les colonnes clés pour simplifier leur manipulation
conso.rename(columns={
    'Date - Heure': 'DateHeure',
    'Date': 'Date',
    'Heure': 'Heure',
    'Consommation brute gaz totale (MW PCS 0°C)': 'ConsommationGazTotale',
    'Consommation brute électricité (MW) - RTE': 'ConsommationElectricite',
    'Consommation brute totale (MW)': 'ConsommationTotale',
    'Consommation brute gaz (MW PCS 0°C) - GRTgaz': 'ConsommationGazGRTgaz',
    'Consommation brute gaz (MW PCS 0°C) - Teréga': 'ConsommationGazTerega',
    'Statut - GRTgaz': 'StatutGRTgaz',
    'Statut - Teréga': 'StatutTerega',
    'Statut - RTE': 'StatutRTE'
}, inplace=True)


# Vérif des nouvelles colonnes
print(conso.columns)


Index(['DateHeure', 'Date', 'Heure', 'ConsommationGazGRTgaz', 'StatutGRTgaz',
       'ConsommationGazTerega', 'StatutTerega', 'ConsommationGazTotale',
       'ConsommationElectricite', 'StatutRTE', 'ConsommationTotale'],
      dtype='object')


##### Gerer les valeurs maquantes

In [47]:
# uniquement les lignes contenant des valeurs NaN  dles colonnes importantes
conso = conso.dropna(subset=['ConsommationGazTotale', 'ConsommationElectricite', 'ConsommationTotale'])
print(conso.info())  # Vérif que NaN ont été supprimés


<class 'pandas.core.frame.DataFrame'>
Index: 108092 entries, 1 to 216191
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DateHeure                108092 non-null  object 
 1   Date                     108092 non-null  object 
 2   Heure                    108092 non-null  object 
 3   ConsommationGazGRTgaz    108092 non-null  float64
 4   StatutGRTgaz             108044 non-null  object 
 5   ConsommationGazTerega    108092 non-null  float64
 6   StatutTerega             107996 non-null  object 
 7   ConsommationGazTotale    108092 non-null  float64
 8   ConsommationElectricite  108092 non-null  int64  
 9   StatutRTE                108092 non-null  object 
 10  ConsommationTotale       108092 non-null  float64
dtypes: float64(4), int64(1), object(6)
memory usage: 9.9+ MB
None


##### colonnes inutiles

In [48]:
conso.drop(columns=['StatutGRTgaz', 'StatutTerega', 'StatutRTE'], inplace=True)
print(conso.info())

<class 'pandas.core.frame.DataFrame'>
Index: 108092 entries, 1 to 216191
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DateHeure                108092 non-null  object 
 1   Date                     108092 non-null  object 
 2   Heure                    108092 non-null  object 
 3   ConsommationGazGRTgaz    108092 non-null  float64
 4   ConsommationGazTerega    108092 non-null  float64
 5   ConsommationGazTotale    108092 non-null  float64
 6   ConsommationElectricite  108092 non-null  int64  
 7   ConsommationTotale       108092 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 7.4+ MB
None


In [49]:
#verif final
print(conso.isnull().sum())

DateHeure                  0
Date                       0
Heure                      0
ConsommationGazGRTgaz      0
ConsommationGazTerega      0
ConsommationGazTotale      0
ConsommationElectricite    0
ConsommationTotale         0
dtype: int64


SAUVEGARDE DES DATA CLEAN DANS UN NEW FILE

In [51]:
os.chdir('/workspaces/projet-iadata')  # Change le répertoire vers la racine
print(os.getcwd())  # Vérif
conso.to_csv('../data/prepared/conso_clean.csv', index=False) 

/workspaces/projet-iadata


OSError: Cannot save file into a non-existent directory: '../data/prepared'